In [3]:
import itertools
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pydotplus
import math
import sys
from sets import Set
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import re 
from graphviz import Digraph
from sklearn import linear_model
from __future__ import division # ensures that default division is real number division
#get_ipython().magic(u'matplotlib inline')
#%matplotlib
mpl.rc('figure', figsize=[10,6]) 

In [4]:
vertex = pd.read_csv('data1/VERTEX.csv')
for i in range(len(vertex)):
    vertex.iloc[i,0] = vertex.iloc[i,0]-1
edge = pd.read_csv('data1/EDGE.csv')
print vertex.shape
print edge.shape

(415, 17)
(488, 7)


In [5]:
vertex.head()
#vertex[vertex['VERTEXID']==117]

,VERTEXID,TYPE,HASH,PIDNAME,PID,PPID,BOOTTIMEUNIX,BOOTTIMESIMPLE,NAME,UID,GID,STARTTIMEUNIX,STARTTIMESIMPLE,COMMANDLINE,PATH,FILENAME,VERSION
0,0,Activity,824758822,System,0,0,9.783140e+11,Sun Dec 31 17:58:18 2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Activity,1543717122,NaN,1,0,NaN,NaN,systemd,0\t0\t0\t0,0\t0\t0\t0,9.783140e+11,Sun Dec 31 17:58:18 2000,/usr/lib/systemd/systemd single,NaN,NaN,NaN
2,2,Activity,-1587681112,NaN,2,0,NaN,NaN,kthreadd,0\t0\t0\t0,0\t0\t0\t0,9.783140e+11,Sun Dec 31 17:58:18 2000,NaN,NaN,NaN,NaN
3,3,Activity,-575549639,NaN,3,2,NaN,NaN,ksoftirqd/0,0\t0\t0\t0,0\t0\t0\t0,9.783140e+11,Sun Dec 31 17:58:18 2000,NaN,NaN,NaN,NaN
4,4,Activity,-1432410094,NaN,4,2,NaN,NaN,kworker/0:0,0\t0\t0\t0,0\t0\t0\t0,9.783140e+11,Sun Dec 31 17:58:18 2000,NaN,NaN,NaN,NaN


In [6]:
edge.head()
edge.iloc[:10,:]

,EDGEID,TYPE,HASH,SRCVERTEXHASH,DSTVERTEXHASH,OPERATION,TIME
0,1,WasInformedBy,-143368052,1543717122,824758822,NaN,NaN
1,2,WasInformedBy,1053681266,-1587681112,824758822,NaN,NaN
2,3,WasInformedBy,-1966253595,-575549639,-1587681112,NaN,NaN
3,4,WasInformedBy,817628190,-1432410094,-1587681112,NaN,NaN
4,5,WasInformedBy,811246560,-1432475884,-1587681112,NaN,NaN
5,6,WasInformedBy,-1088617771,-566501847,-1587681112,NaN,NaN
6,7,WasInformedBy,710637653,-2141961303,-1587681112,NaN,NaN
7,8,WasInformedBy,-1088616510,-566501834,-1587681112,NaN,NaN
8,9,WasInformedBy,817595792,-1432410428,-1587681112,NaN,NaN
9,10,WasInformedBy,-1966108192,-575548140,-1587681112,NaN,NaN


In [7]:
dot = Digraph(comment='graph')
vertex = np.asarray(vertex)
print vertex[117][:5]
edge = np.asarray(edge)
for i in range(len(vertex)):
    dot.node(str(vertex[i][2]),str(vertex[i][0:2]))
for j in range(len(edge)):
    dot.edge(str(edge[j][3]),str(edge[j][4]))

[117 'Entity' -148882091 nan nan]


In [8]:
dot.render('test-output/graph.gv', view=True)

'test-output/graph.gv.pdf'

In [7]:
NodeNum = len(vertex)
EdgeNum = len(edge)
print NodeNum
print EdgeNum
Size = (NodeNum,NodeNum+1)
AdjMatrix = np.zeros(Size)

for k in range(NodeNum):
    AdjMatrix[k][0] = k+1

for i in range(len(edge)):
#for i in range(5):
    for j in range(len(vertex)):
        if (vertex[j][2] == edge[i][3]):      
            startV = vertex[j][0]
            #print "start"
            #print startV
        if(vertex[j][2] == edge[i][4]):
            #print "end"
            endV = vertex[j][0]
            #print endV
    AdjMatrix[endV][startV+1] = 1
#print AdjMatrix
print AdjMatrix[:10,:]

415
488
[[  1.   0.   1. ...,   0.   0.   0.]
 [  2.   0.   0. ...,   0.   0.   0.]
 [  3.   0.   0. ...,   0.   0.   0.]
 ..., 
 [  8.   0.   0. ...,   0.   0.   0.]
 [  9.   0.   0. ...,   0.   0.   0.]
 [ 10.   0.   0. ...,   0.   0.   0.]]


In [36]:
def updateFlag(val,sorted_attr_row):
    for i in range(len(sorted_attr_row)-1):
        val[i] = sorted_attr_row[i+1]

def defineT(attr):
    dtype = [('Id',int)]
    for i in range(len(attr)):
        dtype.append((attr[i],'S10'))
    return dtype

In [37]:
def ACompatible(graph,edge,attr):
    dtype = defineT(attr)
    value = list(graph[attr].itertuples())
    original_table = np.asarray(value,dtype=dtype)
    groupArr = []

    sorted_attr = np.sort(original_table, order=attr)  
    val = ['','']
    for i in range(len(sorted_attr)):
        for j in range(len(attr)):
            if(sorted_attr[i][j+1]==val[j]):
                if(j == len(attr)-1):
                    newGroup.append(sorted_attr[i])
            else:
                newGroup = [(sorted_attr[i])]
                groupArr.append(newGroup)
                updateFlag(val,sorted_attr[i])
                break
    return groupArr

In [38]:
def DataStruture(result1,edge,attr,graph):
    print "update data structure"
    print "groupnumber",len(result1)
    groupNum = len(result1)
    nodeNum = graph.shape[0]
    mapsize = (nodeNum,groupNum)
    bitMap = np.zeros(mapsize)
    PArraySize = (groupNum,groupNum)
    PArray = np.zeros(PArraySize)
    edge = np.asarray(edge)
    
    #initialize bit map
    for i in range(groupNum):
        groupSet = result1[i]
        for j in range(len(groupSet)):
            CurrentNode = groupSet[j]
            #print CurrentNode
            index = CurrentNode[0]
            for m in range(groupNum):
                if(m!=i):
                    groupSet1 = result1[m]
                    for n in range(len(groupSet1)):
                        CurrentNode2 = groupSet1[n]
                        #print CurrentNode
                        index2 = CurrentNode2[0]
                        if(not pd.isnull(edge[index2,index+1])):
                            if(edge[index2,index+1]==1):
                                bitMap[index][m]=1
            

    #print "bitmap length",len(bitMap)
    #print bitMap[117],bitMap[125]

    #initialize participation array
    for i in range(groupNum):
        groupSet = result1[i]
        temp = np.zeros(groupNum)
        for j in range(len(groupSet)):
            CurrentNode = groupSet[j]
            index = CurrentNode[0]
            for k in range(groupNum):
                temp[k] += bitMap[index][k]
        PArray[i] = temp

    #print "parray"
    #print PArray
    return PArray,bitMap

In [39]:
def condition(SubGroup,PArray):
    GroupNum = len(SubGroup);
    for i in range(GroupNum):
        setSize = len(SubGroup[i])
        for j in range(GroupNum):
            if ((PArray[i][j]!=setSize) & (PArray[i][j]!=0.0)):
                return False,i+1
    return True,0
                

In [40]:
def Split(BitMap,fixedGNum,TempResult,graph,attr):
    groupNum = len(TempResult)
    WaitingGroup = TempResult[fixedGNum-1]
    #print "waitinggroup"
    #print WaitingGroup
    tempBitMapsize = (len(WaitingGroup),groupNum)
    tempBitMap = np.zeros(tempBitMapsize)
    NodeIndex = []
    subGroup1 = []
    subGroup2 = []
    
    dtype = defineT(attr)
    value = list(graph[attr].itertuples())
    original_table = np.asarray(value,dtype=dtype)
    #print original_table
    
    for i in range(len(WaitingGroup)):
        NodeIndex.append(WaitingGroup[i][0])
        tempBitMap[i][:] = BitMap[WaitingGroup[i][0]][:]
    #print "nodeindex"
    #print NodeIndex
    #print "temp bit map"
    #print tempBitMap
    #print "node number",len(np.asarray(NodeIndex))
    #print "bit size",len(tempBitMap)
    table = np.concatenate((np.asarray(NodeIndex).reshape(len(np.asarray(NodeIndex)),1),tempBitMap),axis=1)

    typename = []
    for i in range(groupNum):
        typename.append("attr"+str(i))
    
    dtype = [('Id',int)]
    for i in range(groupNum):
        dtype.append((typename[i],float))
    #print dtype

    #print table
    value = []
    for i in range(len(NodeIndex)):
        value.append(tuple(table[i].tolist()))

    waitingSortTable = np.asarray(value,dtype=dtype)
    #print table
    
    for k in range(groupNum):
        temp = np.sort(waitingSortTable, order=[typename[k],'Id']) 
        #print "temp sort attribute"
        #print temp
        for m in range(len(temp)-1):
            if(temp[m][k+1]!=temp[m+1][k+1]):
                #print "found it"
                #print m
                for j in range(m+1):
                    subGroup1.append(original_table[temp[j][0]])
                for n in range(m+1,len(temp)):
                    subGroup2.append(original_table[temp[n][0]])
                #print "group1"
                #print subGroup1
                #print "group2"
                #print subGroup2
                #print TempResult
                TempResult.remove(WaitingGroup)
                TempResult.append(subGroup1)
                TempResult.append(subGroup2)
                #print "result",TempResult
                return TempResult
    return none
        

In [41]:
def SNAP(graph,edge,attr):
    TempResult = ACompatible(graph,edge,attr)
    PArray,BitMap = DataStruture(TempResult,edge,attr,graph)
    #for i in range(len(TempResult)):
    #    print "another group"
    #    print TempResult[i]
    cond,groupNum = condition(TempResult,PArray)
    while(not cond):
    #for j in range(5):
        TempResult = Split(BitMap,groupNum,TempResult,graph,attr)
        PArray,BitMap = DataStruture(TempResult,edge,attr,graph)
        cond,groupNum = condition(TempResult,PArray)
    for i in range(len(TempResult)):
        print "another group"
        print TempResult[i]
    print PArray
    return TempResult,PArray


In [121]:
attr = ['TYPE']
vertex = pd.read_csv('data1/VERTEX.csv')
for i in range(vertex.shape[0]):
    a = vertex.iloc[i]['NAME']
    if((not pd.isnull(a)) & (a!='java') & (a!= 'spade-controlle')):
        vertex.NAME[vertex.VERTEXID == i+1]="system"
    
for i in range(len(vertex)):
    vertex.iloc[i,0] = vertex.iloc[i,0]-1

SMRnode,PArray = SNAP(vertex,AdjMatrix,attr)

update data structure
groupnumber 2
update data structure
groupnumber 3
update data structure
groupnumber 4
update data structure
groupnumber 5
update data structure
groupnumber 6
update data structure
groupnumber 7
update data structure
groupnumber 8
update data structure
groupnumber 9
update data structure
groupnumber 10
update data structure
groupnumber 11
another group
[(117, 'Entity'), (125, 'Entity'), (146, 'Entity'), (249, 'Entity'), (284, 'Entity'), (297, 'Entity'), (304, 'Entity'), (309, 'Entity'), (314, 'Entity'), (326, 'Entity'), (327, 'Entity'), (329, 'Entity'), (337, 'Entity')]
another group
[(247, 'Activity'), (264, 'Activity'), (305, 'Activity')]
another group
[(116, 'Activity'), (145, 'Activity'), (325, 'Activity')]
another group
[(159, 'Activity'), (161, 'Activity'), (165, 'Activity'), (169, 'Activity'), (170, 'Activity'), (179, 'Activity'), (188, 'Activity'), (197, 'Activity'), (206, 'Activity'), (215, 'Activity'), (224, 'Activity'), (233, 'Activity'), (242, 'Activity

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [122]:
dot1 = Digraph(comment='Summary Graph')
#vertex = np.asarray(vertex)
#edge = np.asarray(edge)
#for i in range(len(vertex)):
#    dot.node(str(vertex[i][2]),str(vertex[i][0]))
#for j in range(len(edge)):
#    dot.edge(str(edge[j][3]),str(edge[j][4]))
    
print AdjMatrix[SMRnode[1][0][0],SMRnode[1][0][0]]

for i in range(len(SMRnode)):
    if(i==0):
        color = 'red'
    if(i==1):
        color = 'blue'
    if(i==2):
        color = 'green'
    if(i==3):
        color = 'yellow'
    if(i==4):
        color = 'cyan'
    if(i==5):
        color = 'magenta'
    if(i==6):
        color = 'Purple'
    if(i==7):
        color = 'grey'
    if(i==8):
        color = 'tan'
    if(i==9):
        color = 'SeaGreen'
    if(i==10):
        color = 'chocolate'
    if(i==11):
        color = 'salmon'
    if(i==12):
        color = 'YellowGreen'
    if(i==13):
        color = 'Lavender'
    if(i==14):
        color = 'Olive'
    if(i==15):
        color = 'Orange'
    if(i==16):
        color = 'Violet'
    if(i==17):
        color = 'PeachPuff'
        
    AType = vertex.iloc[SMRnode[i][0][0]]['NAME']
    print AType
    if((not pd.isnull(AType)) & (AType!='java') & (AType!= 'spade-controlle') | (SMRnode[i][0][0]==0)):
        AType = 'System'    
    dot1.node(str(SMRnode[i][0]),str(SMRnode[i][0])+"Type: "+str(AType),color = color,style='filled')
    for j in range(len(SMRnode)):
        if(i!=j):
            if(PArray[i][j]!=0):
                for k in range(int(PArray[i][j])):
                    dot1.edge(str(SMRnode[i][0]),str(SMRnode[j][0]))

dot1.render('test-output/summary_Graph')

0.0
nan
java
java
nan
nan
nan
nan
nan
nan
nan
nan


'test-output/summary_Graph.pdf'

In [119]:
dot2 = Digraph(comment='ColorGraph')
vertex = pd.read_csv('data1/VERTEX.csv')
#vertex = np.asarray(vertex)
edge = np.asarray(edge)
for i in range(len(SMRnode)):
    if(i==0):
        color = 'red'
    if(i==1):
        color = 'blue'
    if(i==2):
        color = 'green'
    if(i==3):
        color = 'yellow'
    if(i==4):
        color = 'cyan'
    if(i==5):
        color = 'magenta'
    if(i==6):
        color = 'Purple'
    if(i==7):
        color = 'grey'
    if(i==8):
        color = 'tan'
    if(i==9):
        color = 'SeaGreen'
    if(i==10):
        color = 'chocolate'
    if(i==11):
        color = 'salmon'
    if(i==12):
        color = 'YellowGreen'
    if(i==13):
        color = 'Lavender'
    if(i==14):
        color = 'Olive'
    if(i==15):
        color = 'Orange'
    if(i==16):
        color = 'Violet'
    if(i==17):
        color = 'PeachPuff'
        
        
    for j in range(len(SMRnode[i])):
        #print SMRnode[i][j]
        hashvalue = vertex.iloc[SMRnode[i][j][0]]['HASH']
        AType = vertex.iloc[SMRnode[i][j][0]]['NAME']
        if((not pd.isnull(AType)) & (AType!='java') & (AType!= 'spade-controlle')):
            AType = 'System'
        dot2.node(str(hashvalue),str(SMRnode[i][j])+str(AType) ,color = color,style='filled')
for j in range(len(edge)):
    dot2.edge(str(edge[j][3]),str(edge[j][4]))
dot2.render('test-output/ColorGrap')

'test-output/ColorGrap.pdf'

In [94]:
vertex = pd.read_csv('data1/VERTEX.csv')
#print vertex.head()
#t = vertex[vertex['VERTEXID']==1]['HASH']
#print t
#print vertex.iloc[0]['HASH']
print vertex.shape

for i in range(vertex.shape[0]):
    a = vertex.iloc[i]['NAME']
    if((not pd.isnull(a)) & (a!='java') & (a!= 'spade-controlle')):
        vertex.NAME[vertex.VERTEXID == i+1]="system"
    print vertex.iloc[i]['NAME']

    


(415, 17)
nan
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
system
java
spade-controlle
java
java
nan
nan
java
java
system
nan
nan
nan
nan
nan
system
system
system
system
system
system
system
system
system
system
system
system
system
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
java
java
nan
nan
nan
nan
nan
nan
nan
java
nan
nan
nan
nan
nan
nan
nan
nan
system
system
system
java
system
system
system
java
system
system
system
java
system
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
java
spade-controlle
nan
na

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [86]:
print vertex.iloc[104]['NAME']
vertex.NAME[vertex.VERTEXID == 6]="system"

print vertex.iloc[6]['NAME']

nan
system


/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
